In [33]:
from ipywidgets import FileUpload
from IPython.display import display
import io

uploader = FileUpload(accept='image/*', multiple=False)
display(uploader)


FileUpload(value=(), accept='image/*', description='Upload')

In [35]:
# Save uploaded file
if uploader.value:
    uploaded_file = uploader.value[0]   # Get the first uploaded file (dictionary inside tuple)
    uploaded_filename = uploaded_file['name']
    uploaded_content = uploaded_file['content']
    
    # Save the uploaded content
    with open('input.png', 'wb') as f:
        f.write(uploaded_content)
    
    print(f"✅ File '{uploaded_filename}' saved as 'input.png'")
else:
    print("❗ No file uploaded. Please upload an image first.")


✅ File 'cat.png' saved as 'input.png'


In [36]:
import subprocess
import os

from transformers import AutoFeatureExtractor, ResNetForImageClassification

username = "zyan319"
remote_host = "login-ice.pace.gatech.edu"
remote_path = "/home/hice1/zyan319/cybershuttle_project"
local_model_path = "./my_local_model"

# Step 0: Check if local my_local_model/ exists
print(f"🔍 Checking if local model folder exists at {local_model_path}...")

if not os.path.exists(local_model_path):
    print("⚡ Local model folder not found. Downloading ResNet-50 model...")

    model_name = "microsoft/resnet-50"
    model = ResNetForImageClassification.from_pretrained(model_name)
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

    os.makedirs(local_model_path, exist_ok=True)
    model.save_pretrained(local_model_path)
    feature_extractor.save_pretrained(local_model_path)

    print("✅ Model downloaded and saved locally.")
else:
    print("✅ Local model folder already exists. Skipping download.")

# Step 1: Check if cybershuttle_project/ exists
print(f"🔍 Checking if project folder exists at {remote_path}...")

check_project = subprocess.run(
    ["ssh", f"{username}@{remote_host}", f"test -d {remote_path}"],
    capture_output=True
)

if check_project.returncode != 0:
    print("⚡ Project folder not found. Creating...")
    subprocess.run(
        ["ssh", f"{username}@{remote_host}", f"mkdir -p {remote_path}"],
        text=True,
        shell=False
    )
    print("✅ Project folder created.")
else:
    print("✅ Project folder already exists.")

# Step 2: Check if myenv/ exists
print(f"🔍 Checking if virtual environment exists at {remote_path}/myenv...")

check_env = subprocess.run(
    ["ssh", f"{username}@{remote_host}", f"test -d {remote_path}/myenv"],
    capture_output=True
)

if check_env.returncode != 0:
    print("⚡ Virtual environment not found. Setting up myenv...")

    setup_env_commands = (
        f"cd {remote_path} && "
        "module load python/3.10 && "
        "python -m venv myenv && "
        "source myenv/bin/activate && "
        "pip install --upgrade pip && "
        "pip install torch torchvision transformers pillow"
    )

    subprocess.run(
        ["ssh", f"{username}@{remote_host}", f"bash -l -c '{setup_env_commands}'"],
        text=True,
        shell=False
    )

    print("✅ Virtual environment created and packages installed.")
else:
    print("✅ Virtual environment already exists. Skipping setup.")

# Step 3: Check if my_local_model/ exists on ICE
print(f"🔍 Checking if model directory exists at {remote_path}/my_local_model...")

check_model = subprocess.run(
    ["ssh", f"{username}@{remote_host}", f"test -d {remote_path}/my_local_model"],
    capture_output=True
)

if check_model.returncode != 0:
    print("⚡ Model not found on ICE. Uploading local my_local_model/ folder...")

    subprocess.run(
        ["scp", "-r", local_model_path, f"{username}@{remote_host}:{remote_path}/my_local_model"],
        text=True,
        shell=False
    )

    print("✅ Model uploaded to ICE HPC.")
else:
    print("✅ Model already exists on ICE. Skipping upload.")


🔍 Checking if local model folder exists at ./my_local_model...
✅ Local model folder already exists. Skipping download.
🔍 Checking if project folder exists at /home/hice1/zyan319/cybershuttle_project...
✅ Project folder already exists.
🔍 Checking if virtual environment exists at /home/hice1/zyan319/cybershuttle_project/myenv...
✅ Virtual environment already exists. Skipping setup.
🔍 Checking if model directory exists at /home/hice1/zyan319/cybershuttle_project/my_local_model...
✅ Model already exists on ICE. Skipping upload.


In [37]:
import subprocess
import os

username = "zyan319"
remote_host = "login-ice.pace.gatech.edu"
remote_path = "/home/hice1/zyan319/cybershuttle_project"

# Step 0: Upload job_script.sh to remote
print("📤 Uploading job_script.sh to ICE...")

upload_result = subprocess.run(
    ["scp", "job_script.sh", "run_model.py", "input.png", f"{username}@{remote_host}:{remote_path}/"],
    capture_output=True,
    text=True
)

if upload_result.returncode == 0:
    print("✅ job_script.sh uploaded successfully.")
else:
    print("❗ Failed to upload job_script.sh. See error:")
    print(upload_result.stderr)
    raise RuntimeError("Failed to upload job script.")

# Step 1: Submit the job via SSH
print("🚀 Submitting job...")

result = subprocess.run(
    ["ssh", f"{username}@{remote_host}", f"cd {remote_path} && sbatch job_script.sh"],
    capture_output=True,
    text=True
)

# Print output for debugging
print("STDOUT:")
print(result.stdout)

print("STDERR:")
print(result.stderr)

# Step 2: Check if job submission succeeded
if "Submitted batch job" in result.stdout.strip():
    job_id = result.stdout.strip().split()[-1]
    print(f"🚀 Job ID submitted: {job_id}")
else:
    print("❗ sbatch did not submit correctly. Please check error above.")


📤 Uploading job_script.sh to ICE...
✅ job_script.sh uploaded successfully.
🚀 Submitting job...
STDOUT:
Submitted batch job 2540110

STDERR:

🚀 Job ID submitted: 2540110


In [38]:
import time

# Polling: wait for job to disappear from the queue
while True:
    queue_check = subprocess.run(
        ["ssh", f"{username}@{remote_host}", f"squeue -u {username} | grep {job_id}"],
        capture_output=True,
        text=True
    )
    
    if queue_check.stdout == "":
        print("✅ Job completed!")
        break
    else:
        print("⏳ Job still running...waiting 10 seconds...")
        time.sleep(10)


⏳ Job still running...waiting 10 seconds...
⏳ Job still running...waiting 10 seconds...
⏳ Job still running...waiting 10 seconds...
⏳ Job still running...waiting 10 seconds...
✅ Job completed!


In [39]:
import os

# Download output.txt and error.log back to local
subprocess.run(["scp", f"{username}@{remote_host}:{remote_path}/output.txt", "."])
subprocess.run(["scp", f"{username}@{remote_host}:{remote_path}/error.log", "."])
print("✅ Files downloaded.")
# Step 2: Delete output.txt and error.log on remote server
print("🧹 Cleaning up remote output files...")

subprocess.run(["ssh", f"{username}@{remote_host}", f"rm -f {remote_path}/output.txt {remote_path}/error.log"])


print("📄 Prediction Result:")
with open('output.txt', 'r') as f:
    result_content = f.read()
print(result_content)


✅ Files downloaded.
🧹 Cleaning up remote output files...
📄 Prediction Result:
Predicted class: tabby, tabby cat
Execution time: 0.3223 seconds
Peak memory usage: 2.5052 MB

